In [114]:
from dcd.entities.thing import Thing
from dotenv import load_dotenv
import os

# Connect to DCD Hub
load_dotenv()

THING_ID = os.environ['THING_ID']
THING_TOKEN = os.environ['THING_TOKEN']
my_thing = Thing(thing_id=THING_ID, token=THING_TOKEN)

my_thing.read()
my_thing.to_json()

INFO:dcd:things:wheelchair_speed-e706:Initialising MQTT connection for Thing 'dcd:things:wheelchair_speed-e706'
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dwd.tudelft.nl:443
INFO:dcd:things:wheelchair_speed-e706:Connection successful
DEBUG:urllib3.connectionpool:https://dwd.tudelft.nl:443 "GET /api/things/dcd:things:wheelchair_speed-e706 HTTP/1.1" 200 6724


DigiCertCA.crt exist.
{'thing': {'id': 'dcd:things:wheelchair_speed-e706', 'name': 'Wheelchair_speed', 'description': '', 'type': 'Speed', 'properties': [{'type': 'THREE_DIMENSIONS', 'name': 'My Random Property', 'description': '', 'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''}, {'name': 'Value2', 'description': '', 'unit': ''}, {'name': 'Value3', 'description': '', 'unit': ''}, {'name': 'Value1', 'description': '', 'unit': ''}, {'name': 'Value2', 'description': '', 'unit': ''}, {'name': 'Value3', 'description': '', 'unit': ''}], 'id': 'my-random-property-5f3c', 'classes': [], 'values': [], 'entityId': 'dcd:things:wheelchair_speed-e706', 'readAt': 1572620569665, 'registeredAt': 1571392699000}, {'type': 'ONE_DIMENSION', 'name': 'one', 'description': '', 'dimensions': [{'name': 'Value', 'description': '', 'unit': ''}], 'id': 'one-28d7', 'classes': [], 'values': [], 'entityId': 'dcd:things:wheelchair_speed-e706', 'readAt': 1572620569665, 'registeredAt': 1571994791000}, {

{'id': 'dcd:things:wheelchair_speed-e706',
 'name': 'Wheelchair_speed',
 'description': '',
 'type': 'Speed',
 'properties': [{'id': 'my-random-property-5f3c',
   'name': 'My Random Property',
   'description': '',
   'type': 'THREE_DIMENSIONS',
   'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Value3', 'description': '', 'unit': ''},
    {'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Value3', 'description': '', 'unit': ''}]},
  {'id': 'one-28d7',
   'name': 'one',
   'description': '',
   'type': 'ONE_DIMENSION',
   'dimensions': [{'name': 'Value', 'description': '', 'unit': ''}]},
  {'id': 'random-shit-aa03',
   'name': 'RANDOM SHIT',
   'description': '',
   'type': 'THREE_DIMENSIONS',
   'dimensions': [{'name': 'Value1', 'description': '', 'unit': ''},
    {'name': 'Value2', 'description': '', 'unit': ''},
    {'name': 'Val

In [115]:
from datetime import datetime, timedelta
# What dates?
now = datetime.now()
now_minus_10 = now + timedelta(minutes = -1)

START_DATE = datetime.strftime(now_minus_10, "%Y-%m-%d %H:%M:%S")
END_DATE = datetime.strftime(now, "%Y-%m-%d %H:%M:%S")

DATE_FORMAT = '%Y-%m-%d %H:%M:%S'
from_ts = datetime.timestamp(datetime.strptime(START_DATE, DATE_FORMAT)) * 1000
to_ts = datetime.timestamp(datetime.strptime(END_DATE, DATE_FORMAT)) * 1000

PROPERTY_NAME = "Speed"

my_property = my_thing.find_property_by_name(PROPERTY_NAME)
my_property.read(from_ts, to_ts)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): dwd.tudelft.nl:443
DEBUG:urllib3.connectionpool:https://dwd.tudelft.nl:443 "GET /api/things/dcd:things:wheelchair_speed-e706/properties/speed-83ac?from=1572620509000.0&to=1572620569000.0 HTTP/1.1" 200 321


In [116]:
print(len(my_property.values))
my_property.values

0


[]

In [117]:
#Create CSV file from data 
import pandas as pd

ct = datetime.now().strftime('%A %-d %B %Y %H:%M')

data = my_property.values
df = pd.DataFrame(my_property.values, columns = ['time', 'velocity'])
df.to_csv('Workout '+ct+'.csv')
print(ct)

Friday 1 November 2019 16:02


In [118]:
#update Rootfolder.csv
df_root = pd.read_csv('Rootfolder.csv')
newdf_root = pd.DataFrame({'filename' : ['Workout '+ct+'.csv']})
UPDATE=df_root.append(newdf_root, ignore_index=True)
UPDATE.to_csv('Rootfolder.csv', index=False)


In [119]:
#Display different plots
import ipywidgets as widgets
from ipywidgets import interactive, interact, interact_manual
import plotly.express as px

df_root = pd.read_csv('Rootfolder.csv')

def selected_workout(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, "workout")
    return unique

dropdown_file = widgets.Dropdown(options = selected_workout(df_root.filename))

output_file = widgets.Output()

def dropdown_file_eventhandler(change):
    output_file.clear_output()
    with output_file:
        if (change.new=="Workout"):
            print('select your workout')
        else:
            global df_plot
            df_plot=pd.read_csv(change.new)
            fig1 = px.line(df_plot, x = 'time', y = 'velocity', title='Workout ')
            fig1.show()

dropdown_file.observe(dropdown_file_eventhandler, names='value')

display(dropdown_file)


Dropdown(options=('workout', 'Workout Friday 1 November 2019 15:24.csv', 'Workout Friday 1 November 2019 15:26…

In [120]:
display(output_file)

Output()

In [121]:
#show statistics
import numpy as np
from scipy.stats import kurtosis, skew

